In [ ]:
# Question 10
import cv2
import numpy as np
import matplotlib.pyplot as plt

def analyze_sapphires_advanced(image_path, focal_length_mm=8, distance_mm=480, pixel_pitch_mm=0.0048):
    """
    Q10: Advanced sapphire analysis with better segmentation and accurate area calculation
    """
    # Read image
    bgr = cv2.imread(image_path)
    if bgr is None:
        raise ValueError(f"Could not load image from {image_path}")
    
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    
    # (a) Apply segmentation algorithm in HSV space
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    
    # Blue color range for sapphires (adjusted based on actual image)
    lower_blue = np.array([100, 60, 40], dtype=np.uint8)   # Hue, Sat, Value
    upper_blue = np.array([140, 255, 255], dtype=np.uint8)
    
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # Morphological cleanup
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # (b) Fill holes completely
    def fill_holes(binary_mask):
        """Complete hole filling using flood fill approach"""
        inverted = cv2.bitwise_not(binary_mask)
        h, w = inverted.shape
        flood_mask = np.zeros((h + 2, w + 2), np.uint8)
        flood_filled = inverted.copy()
        cv2.floodFill(flood_filled, flood_mask, (0, 0), 255)
        holes = cv2.bitwise_not(flood_filled)
        return cv2.bitwise_or(binary_mask, holes)
    
    filled_mask = fill_holes(mask)
    
    # Keep only the two largest components (sapphires)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(filled_mask, connectivity=8)
    
    if num_labels < 3:  # Background + at least 2 sapphires
        raise ValueError("Not enough components found - check segmentation")
    
    # Get two largest components (skip background label 0)
    areas = [(i, stats[i, cv2.CC_STAT_AREA]) for i in range(1, num_labels)]
    areas_sorted = sorted(areas, key=lambda x: x[1], reverse=True)[:2]
    keep_indices = [i for i, _ in areas_sorted]
    
    final_mask = np.isin(labels, keep_indices).astype(np.uint8) * 255
    final_mask = fill_holes(final_mask)  # Final hole filling
    
    # (c) Get areas in pixels
    num_final, labels_final, stats_final, centroids_final = cv2.connectedComponentsWithStats(final_mask, connectivity=8)
    sapphire_areas_px = [int(stats_final[i, cv2.CC_STAT_AREA]) for i in range(1, num_final)]
    
    # (d) Compute actual areas using pinhole camera model
    mm_per_px = (distance_mm / focal_length_mm) * pixel_pitch_mm
    mm2_per_px = mm_per_px ** 2
    sapphire_areas_mm2 = [round(area * mm2_per_px, 2) for area in sapphire_areas_px]
    sapphire_areas_cm2 = [round(area_mm2 / 100, 4) for area_mm2 in sapphire_areas_mm2]
    
    return (rgb, hsv, mask, filled_mask, final_mask, 
            sapphire_areas_px, sapphire_areas_mm2, sapphire_areas_cm2,
            mm_per_px, mm2_per_px, focal_length_mm, distance_mm, pixel_pitch_mm)

# Main execution
def main():
    image_path = 'E:/UoM MSc in AI/Semester 3/IT5437 - Computer Vision/Assignment/a1images/sapphire.jpg'
    
    # Camera parameters (given in question)
    focal_length_mm = 8.0    # f = 8mm
    distance_mm = 480.0      # 480mm from table surface
    pixel_pitch_mm = 0.0048  # Typical pixel size (4.8μm)
    
    try:
        print("=" * 80)
        print("QUESTION 10 - SAPPHIRE MEASUREMENT (ADVANCED)")
        print("=" * 80)
        
        # Analyze sapphires with advanced method
        (rgb, hsv, mask, filled_mask, final_mask, 
         areas_px, areas_mm2, areas_cm2,
         mm_per_px, mm2_per_px, f, d, pixel_pitch) = analyze_sapphires_advanced(
            image_path, focal_length_mm, distance_mm, pixel_pitch_mm
        )
        
        # Create comprehensive visualization
        plt.figure(figsize=(20, 12))
        
        # Row 1: Original image and color spaces
        plt.subplot(3, 4, 1)
        plt.imshow(rgb)
        plt.title('Original Image\nSapphires on Table', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(3, 4, 2)
        plt.imshow(hsv[:,:,0], cmap='hsv')
        plt.title('HSV - Hue Channel', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(3, 4, 3)
        plt.imshow(hsv[:,:,1], cmap='gray')
        plt.title('HSV - Saturation Channel', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(3, 4, 4)
        plt.imshow(hsv[:,:,2], cmap='gray')
        plt.title('HSV - Value Channel', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        # Row 2: Segmentation pipeline
        plt.subplot(3, 4, 5)
        plt.imshow(mask, cmap='gray')
        plt.title('(a) Initial Segmentation\nColor-based Mask', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(3, 4, 6)
        plt.imshow(filled_mask, cmap='gray')
        plt.title('(b) After Hole Filling\nFlood Fill Method', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(3, 4, 7)
        plt.imshow(final_mask, cmap='gray')
        plt.title('(b) Final Mask\nTwo Largest Components', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(3, 4, 8)
        overlay = rgb.copy()
        overlay[final_mask == 0] = (overlay[final_mask == 0] * 0.3).astype(np.uint8)
        plt.imshow(overlay)
        plt.title('Final Segmentation Overlay', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        # Row 3: Results and analysis
        plt.subplot(3, 4, 9)
        # Individual sapphire masks
        num_sapphires = len(areas_px)
        colors = plt.cm.Blues(np.linspace(0.3, 0.8, num_sapphires))
        
        for i in range(num_sapphires):
            plt.bar(i, areas_px[i], color=colors[i], alpha=0.7, label=f'Sapphire {i+1}')
        
        plt.title('(c) Pixel Areas', fontweight='bold', fontsize=10)
        plt.xlabel('Sapphire')
        plt.ylabel('Area (pixels)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(3, 4, 10)
        for i in range(num_sapphires):
            plt.bar(i, areas_mm2[i], color=colors[i], alpha=0.7, label=f'Sapphire {i+1}')
        
        plt.title('(d) Actual Areas (mm²)', fontweight='bold', fontsize=10)
        plt.xlabel('Sapphire')
        plt.ylabel('Area (mm²)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(3, 4, 11)
        # Camera geometry diagram
        plt.text(0.1, 0.8, 'Camera Geometry:', fontsize=12, fontweight='bold')
        plt.text(0.1, 0.6, f'Focal length (f) = {f} mm', fontsize=10)
        plt.text(0.1, 0.4, f'Distance (Z) = {d} mm', fontsize=10)
        plt.text(0.1, 0.2, f'Pixel pitch = {pixel_pitch} mm', fontsize=10)
        plt.axis('off')
        
        plt.subplot(3, 4, 12)
        # Conversion factors
        plt.text(0.1, 0.8, 'Conversion Factors:', fontsize=12, fontweight='bold')
        plt.text(0.1, 0.6, f'{mm_per_px:.6f} mm/pixel', fontsize=10)
        plt.text(0.1, 0.4, f'{mm2_per_px:.6f} mm²/pixel', fontsize=10)
        plt.text(0.1, 0.2, f'Z/f = {d/f:.1f}', fontsize=10)
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

        print(f"\n(a) Segmentation algorithm:")
        print("   - HSV color space segmentation (Hue: 100-140)")
        print("   - Morphological opening/closing for noise removal")
        print("   - Blue color range selected for sapphires")
        
        print(f"\n(b) Morphological operations and hole filling:")
        print("   - Flood fill algorithm for complete hole filling")
        print("   - Selection of two largest connected components")
        print("   - Final hole filling for solid sapphire blobs")
        
        print(f"\n(c) Connected components areas in pixels:")
        for i, area_px in enumerate(areas_px):
            print(f"   Sapphire {i+1}: {area_px} pixels")
        
        print(f"\n(d) Actual area calculation (f={f}mm, Z={d}mm):")
        print("   Using pinhole camera model:")
        print("   mm_per_pixel = (Z / f) × pixel_pitch")
        print("   mm²_per_pixel = (mm_per_pixel)²")
        print("   actual_area = pixel_area × mm²_per_pixel")
        print(f"   pixel_pitch = {pixel_pitch} mm (assumed)")
        print(f"   mm_per_pixel = {mm_per_px:.6f} mm/px")
        print(f"   mm²_per_pixel = {mm2_per_px:.6f} mm²/px")
        print()
        
        for i, (area_px, area_mm2, area_cm2) in enumerate(zip(areas_px, areas_mm2, areas_cm2)):
            print(f"   Sapphire {i+1}:")
            print(f"   - Pixel area: {area_px} px")
            print(f"   - Actual area: {area_mm2} mm²")
            print(f"   - Actual area: {area_cm2} cm²")
        
        print(f"\n" + "=" * 80)
        print("KEY IMPROVEMENTS")
        print("=" * 80)
        print("1. Better hole filling using flood fill algorithm")
        print("2. HSV color space for more robust segmentation")
        print("3. Selection of only two largest components")
        print("4. Proper pinhole camera model implementation")
        print("5. Comprehensive visualization and analysis")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()